In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cv2

plt.rc('image', cmap='gray') # set the default color-map of plt.image() to 'gray'
plt.rc('figure', figsize=[8,8]) # set a bigger default figure size

## playing the video as is

In [ ]:
# cap = cv2.VideoCapture('P4_juggling.mov')

# while True:
#     ret, frame = cap.read() # read next frame
#     if ret==True:
#         frame = cv2.resize(frame, (640, 360)) 
#         cv2.imshow('WINDOW_NAME', frame)
        
#         if (cv2.waitKey(1) & 0xff == 27): # ESC key pressed?
#             break
#     else:
#         break

# cap.release() # release input video
# cv2.destroyAllWindows() # delete output window
# cv2.waitKey(1);

# Contours 
## Red 

In [ ]:
def buildRedMask(frame):
    lowerRed = np.array([0,27,217])
    upperRed = np.array([14,206,255])
    
    frame = cv2.resize(frame, (640, 360)) 
    blur = cv2.GaussianBlur(frame ,(0,0),5)
    hsvRed = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    maskRed = cv2.inRange(hsvRed, lowerRed, upperRed)
    return maskRed

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')
# Take first frame and find corners in it
ret, frame = cap.read()

maskRed = buildRedMask(frame)

blank = np.zeros(frame.shape,dtype='uint8')
contours, _ = cv2.findContours(maskRed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
cv2.drawContours(blank,contours,-1,(255,0,0),1)

plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.imshow(blank)
plt.title('Red contours')

plt.subplot(1,2,2)
plt.imshow(maskRed)
plt.title('Red ball hsv mask')

plt.show()

### Red ball overlay

In [ ]:
redBall = contours[0]
(x,y),radiusRed = cv2.minEnclosingCircle(redBall)
center = (int(x)-4,int(y))
radiusRed = int(radiusRed+3)
cv2.circle(frame,center,radiusRed,(255,255,255),2, lineType=cv2.LINE_AA)

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

plt.imshow(frame)

### Tracking the red ball

In [ ]:
def redDraw(redMask):
    contours, _ = cv2.findContours(redMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    redBall = contours[0]    
    (x,y),_ = cv2.minEnclosingCircle(redBall)
    center = (int(x),int(y))    
    cv2.circle(frame,center,radiusRed,(255,255,255),2,lineType=cv2.LINE_AA)
    cv2.imshow('Red-Track',frame) # display frame in a window
    
    return (x,y)

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')

while(True):
    ret ,frame = cap.read()
    if ret == True:
    
        redMask = buildRedMask(frame)
        (x,y) = redDraw(redMask)
#         cv2.imshow('Red-Track',frame) # display frame in a window

        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
    else:
        break

cap.release() # release input video
cv2.destroyAllWindows() # delete output window
cv2.waitKey(1);

## Blue

In [ ]:
def buildBlueMask(frame):
    lowerBlue = np.array([80,95,95])
    upperBlue = np.array([179,255,255])

    frame = cv2.resize(frame, (640, 360)) 
    blur = cv2.GaussianBlur(frame ,(0,0),5)
    hsvBlue = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    maskBlue = cv2.inRange(hsvBlue, lowerBlue, upperBlue)
    return maskBlue

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')
# Take first frame and find corners in it
ret, frame = cap.read()

maskBlue = buildBlueMask(frame)

blank = np.zeros(frame.shape,dtype='uint8')
contours, _ = cv2.findContours(maskBlue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
cv2.drawContours(blank,contours,-1,(255,0,0),1)

plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.imshow(blank)
plt.title('Blue contours')

plt.subplot(1,2,2)
plt.imshow(maskBlue)
plt.title('Blue ball hsv mask')

plt.show()

### Blue ball overlay

In [ ]:
blueBall = contours[0]
(x,y),radiusBlue = cv2.minEnclosingCircle(blueBall)
center = (int(x),int(y))
radiusBlue = int(radiusBlue+1)
cv2.circle(frame,center,radiusBlue,(255,255,255),2, lineType=cv2.LINE_AA)

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(frame)

### Tracking the blue ball

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')
ret ,frame = cap.read()

blueMask = buildBlueMask(frame)
contours, _ = cv2.findContours(blueMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
print(len(contours))

sizes = []
for i in range(len(contours)):
    M = cv2.moments(contours[i])
    sizes.append(M['m00'])

print(np.unique(sizes).astype(np.int64))

ballContours = []
for i in range(len(contours)):
    M = cv2.moments(contours[i])
    if 0<(M['m00'])<100:
        ballContours.append(contours[i])
        
print(len(ballContours))

In [ ]:
def blueDraw(blueMask):          
    contours, _ = cv2.findContours(blueMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    ballContours = []
    for i in range(len(contours)):
        M = cv2.moments(contours[i])
        if (M['m00'])>100:
            ballContours.append(contours[i])
            blueBall = ballContours[0]
            (x,y),_ = cv2.minEnclosingCircle(blueBall)
            center = (int(x),int(y))    
            cv2.circle(frame,center,radiusBlue,(255,255,255),2, lineType=cv2.LINE_AA)
            cv2.imshow('Blue',frame) # display frame in a window
    return (x,y)            

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')

while(True):
    ret ,frame = cap.read()
    if ret == True:
        
        blueMask = buildBlueMask(frame)
#         contours, _ = cv2.findContours(blueMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
#         ballContours = []
#         for i in range(len(contours)):
#             M = cv2.moments(contours[i])
#             if (M['m00'])>100:
#                 ballContours.append(contours[i])
#                 blueBall = ballContours[0]
#                 (x,y),_ = cv2.minEnclosingCircle(blueBall)
#                 center = (int(x),int(y))    
#                 cv2.circle(frame,center,radiusBlue,(255,255,255),2, lineType=cv2.LINE_AA)
        (x,y) = blueDraw(blueMask)

#         cv2.imshow('Blue',frame) # display frame in a window

        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
    else:
        break

cap.release() # release input video
cv2.destroyAllWindows() # delete output window
cv2.waitKey(1);

## green

In [ ]:
def buildGreenMask(frame): 
    lowerGreen = np.array([42,30,35])
    upperGreen = np.array([70,255,255])
    frame = cv2.resize(frame, (640, 360)) 
#     blur = cv2.GaussianBlur(frame ,(0,0),3)
    blur = cv2.GaussianBlur(frame ,(0,0),1)
    hsvGreen = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    maskGreen = cv2.inRange(hsvGreen, lowerGreen, upperGreen)
    return maskGreen

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')
# Take first frame and find corners in it
ret, frame = cap.read()

maskGreen = buildGreenMask(frame) 

blank = np.zeros(frame.shape,dtype='uint8')
contours, _ = cv2.findContours(maskGreen, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
cv2.drawContours(blank,contours,-1,(255,0,0),1)

plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.imshow(blank)
plt.title('Green contours')

plt.subplot(1,2,2)
plt.imshow(maskGreen)
plt.title('Green ball hsv mask')

plt.show()

### Green ball overlay

In [ ]:
greenBall = contours[0]
(x,y),radiusGreen = cv2.minEnclosingCircle(greenBall)
center = (int(x),int(y))
radiusGreen = int(radiusGreen)
cv2.circle(frame,center,radiusGreen,(255,255,255),2, lineType=cv2.LINE_AA)

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(frame)

### Tracking the green ball

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')
ret ,frame = cap.read()

mask = buildGreenMask(frame)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
print(len(contours))

sizes = []
for i in range(len(contours)):
    M = cv2.moments(contours[i])
    sizes.append(M['m00'])

print(np.unique(sizes).astype(np.int64))

In [ ]:
def greenDraw(greenMask):
    contours, _ = cv2.findContours(greenMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    ballContours = []
    for i in range(len(contours)):
        M = cv2.moments(contours[i])
        if (M['m00'])>100:
            ballContours.append(contours[i])
            greenBall = ballContours[0]
            (x,y),_ = cv2.minEnclosingCircle(greenBall)
            center = (int(x),int(y))    
            cv2.circle(frame,center,radiusGreen,(255,255,255),2,lineType=cv2.LINE_AA)
            cv2.imshow('Green-Track',frame) # display frame in a window       
    return (x,y)

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')

while(True):
    ret ,frame = cap.read()
    if ret == True:
        
        greenMask = buildGreenMask(frame)
        
#         contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
#         ballContours = []
#         for i in range(len(contours)):
#             M = cv2.moments(contours[i])
#             if (M['m00'])>100:
#                 ballContours.append(contours[i])
#                 greenBall = ballContours[0]
#                 (x,y),_ = cv2.minEnclosingCircle(greenBall)
#                 center = (int(x),int(y))    
#                 cv2.circle(frame,center,radiusGreen,(255,255,255),2, lineType=cv2.LINE_AA)
#                 cv2.imshow('Green-Track',frame) # display frame in a window

        (x,y) = greenDraw(greenMask)

        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
    else:
        break

cap.release() # release input video
cv2.destroyAllWindows() # delete output window
cv2.waitKey(1)

# Red & Blue & Green - Tracking

In [ ]:
def build_mask(frame):
    redMask = buildRedMask(frame)
    blueMask = buildBlueMask(frame)
    greenMask = buildGreenMask(frame)
    return redMask, blueMask, greenMask

In [ ]:
def draw_overlay(redMask, blueMask, greenMask):
    (x_red,y_red) = redDraw(redMask) 
#     (x_blue,y_blue) = blueDraw(blueMask)
    (x_green,y_green) = greenDraw(greenMask)
    
    return (x_red,y_red), (x_green,y_green)    
#     return (x_red,y_red), (x_blue,y_blue), (x_green,y_green)

## Overlay trajectories

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')

# Take first frame and find corners in it
ret, frame = cap.read()
if ret == True: 
    redMask, blueMask, greenMask = build_mask(frame)
    (x_red,y_red), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)
#     (x_red,y_red), (x_blue,y_blue), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)

canvas = np.zeros_like(frame)
canvas = np.uint8(canvas*0.95) # fade out canvas  

while(True):
    ret ,frame = cap.read()
    if ret == True: 
        redMask, blueMask, greenMask = build_mask(frame)

        (old_X_red,old_Y_red) = (x_red,y_red)
#         (old_X_blue,old_Y_blue) = (x_blue,y_blue)
        (old_X_green,old_Y_green) = (x_green,y_green)
        
        (x_red,y_red), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)
#         (x_red,y_red), (x_blue,y_blue), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)
        
        line_red = (0,0,255)
        line_blue = (255,0,0)
        line_green = (0,255,0)
        
        f1 = cv2.line(canvas,(int(old_X_red),int(old_Y_red)),(int(x_red),int(y_red)),line_red,3)
#         f2 = cv2.line(canvas,(int(old_X_blue),int(old_Y_blue)),(int(x_blue),int(y_blue)),line_blue,3)
        f3 = cv2.line(canvas,(int(old_X_green),int(old_Y_green)),(int(x_green),int(y_green)),line_green,3)

        frame = cv2.add(f1, frame)
#         frame = cv2.add(f2, frame)
        frame = cv2.add(f3, frame)
        
        cv2.imshow('overlay trajectories',frame)
        cv2.imshow('canvas', canvas)
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
    else:
        break

cap.release() # release input video
cv2.destroyAllWindows() # delete output window
cv2.waitKey(1)

## Velocity

In [ ]:
def calcDist(x1, y1, x2, y2):
    xHold = x1 - x2
    yHold = y1 - y2
    xSquare = xHold ** 2
    ySquare = yHold ** 2
    return (xSquare + ySquare) ** 0.5

In [ ]:
def putVelocity(img, text, org):
    rect_color = (240,240,240)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_color = (10,10,10)
    font_size = 0.5
    thickness = cv2.FILLED
    margin = 3

    text_size = cv2.getTextSize(text, font, font_size, thickness)
    end_x = org[0] + text_size[0][0] + margin
    end_y = org[1] - text_size[0][1] - margin

    cv2.rectangle(img, org, (end_x, end_y), rect_color, thickness)
    cv2.putText(img, text, org, font, font_size, font_color, 1, cv2.LINE_AA)

In [ ]:
cap = cv2.VideoCapture('P4_juggling.mov')

fps = (cap.get(cv2.CAP_PROP_FPS) / 100)

velocity_red_arr = []
velocity_blue_arr = []
velocity_green_arr = []

# Take first frame and find corners in it
ret, frame = cap.read()
if ret == True: 
    redMask, blueMask, greenMask = build_mask(frame)
    (x_red,y_red), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)
#     (x_red,y_red), (x_blue,y_blue), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)

while(True):
    ret ,frame = cap.read()
    if ret == True: 
        redMask, blueMask, greenMask = build_mask(frame)

        (old_X_red,old_Y_red) = (x_red,y_red)
#         (old_X_blue,old_Y_blue) = (x_blue,y_blue)
        (old_X_green,old_Y_green) = (x_green,y_green)
        
        (x_red,y_red), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)
#         (x_red,y_red), (x_blue,y_blue), (x_green,y_green) = draw_overlay(redMask, blueMask, greenMask)
        
        VelocityRed = '%.2f' % (calcDist(old_X_red,old_Y_red,x_red,y_red) / fps)
#         VelocityBlue = '%.2f' % (calcDist(old_X_blue,old_Y_blue,x_blue,y_blue) / fps)
        VelocityGreen = '%.2f' % (calcDist(old_X_green,old_Y_green,x_green,y_green) / fps)

        velocity_red_arr.append(VelocityRed)
#         velocity_blue_arr.append(VelocityBlue)
        velocity_green_arr.append(VelocityGreen)
        
        putVelocity(frame, VelocityRed,(int(x_red+15),int(y_red+5)))
#         putVelocity(frame, VelocityBlue,(int(x_blue+15),int(y_blue+5)))
        putVelocity(frame, VelocityGreen,(int(x_green+15),int(y_green+5)))

        cv2.imshow('Track-all',frame) # display frame in a window

        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
    else:
        break

cap.release() # release input video
cv2.destroyAllWindows() # delete output window
cv2.waitKey(1)

### Velocity of each ball, as a function of time

In [ ]:
v_red = []
for i in velocity_red_arr:
    v_red.append(float(i))

plt.figure(figsize=(20,4))
plt.plot(v_red,'.-');
plt.title("Velocity of red ball")
plt.ylabel("Velocity")
plt.xlabel("Time")
plt.grid()
plt.show()

In [ ]:
v_blue = []
for i in velocity_blue_arr:
    v_blue.append(float(i))

plt.figure(figsize=(20,4))
plt.plot(v_blue,'.-');
plt.title("Velocity of blue ball")
plt.ylabel("Velocity")
plt.xlabel("Time")
plt.grid()
plt.show()

In [ ]:
v_green = []
for i in velocity_green_arr:
    v_green.append(float(i))

plt.figure(figsize=(20,4))
plt.plot(v_green,'.-');
plt.title("Velocity of blue ball")
plt.ylabel("Velocity")
plt.xlabel("Time")
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20,4))

plt.plot(v_blue,'.-',label = "Blue Ball");
plt.plot(v_red,'.-',label = "Red Ball");
plt.plot(v_green,'.-',label = "Green Ball");
  
plt.ylabel('Velocity') 
plt.xlabel('time') 
plt.title('Velocity of each ball, as a function of time') 
plt.legend() 
plt.show() 

## End is here